## Convolving light-curves 

The script convolves ligth curves to get the microlensing signal.

1. Setting the path for the directories and input parameters
2. Function to calculate the difference between A, B images
3. Functions to convolve the data
4. Execution of convolving the maps and saving the data

Rewritten by: Soumya Shreeram <br>
Script originally written by: Eric Paic <br>
Date: 02nd March 2020 <br>

In [1]:
import numpy as np
from astropy.io import fits
import os,sys
from astropy.convolution import convolve_fft
import scipy.signal as ss
#from guppy import hpy

### 1. Setting the path for the directories and input parameters

In [34]:
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_dir = os.path.join(root_dir, "TP4b")
print("Does the directory exist? \n>",os.path.isdir(data_dir))

# setting the paths
datadir = os.path.join(data_dir,  'Data')
resultdir = os.path.join(datadir,  'results')
mapdir = os.path.join(datadir,  'maps', 'unconvolved')
storagedir = os.path.join(datadir,  'maps', 'storage')

Does the directory exist? 
> True


All input parameters:

In [9]:
list_r0 = [2,4,10,15,20,30,40,60,80,100]

list_img = ['A1','B1','A2','B2','A3','B3','A4','B4']

list_comb = [('A1', 'B2'),('A2','B1'),('A3','B3'),('A4','B3'),('A2','B4'),('A1','B3'),('A6','B2'),('A6','B3'),('A7','B3'),('A8','B2'),('A8','B3')]

einstein_r = 3.414e16
cm_per_pxl = (20*einstein_r)/8192
ld_per_pxl = 30000000000*3600*24/cm_per_pxl

dim = 512

### 2. Function to calculated the difference between the A, B images

In [10]:
def mapDiff(mapA,mapB,r0,comb):
    """
    Function takes the logarithmic difference between the two maps A and B
    Input:
    @mapA, mapB :: two randomly chosen maps
    @r0 :: scale radius
    @comb :: combination of the two maps eg. A1, B2 or A3, B2, etc.
    """
    img = fits.open(mapA)[0]
    map_A = img.data[:, :]

    img = fits.open(mapB)[0]
    map_B = img.data[:, :]

    final_map = map_A/map_B

    hdu = fits.PrimaryHDU(final_map)
    os.chdir(data_dir)
    hdu.writeto(resultdir+'/map%s-%s_fml09_R%s_thin_disk.fits'%(comb[0],comb[1],r0))
    return final_map

### 3. Functions to convolve the data

The models of the light source used to convolve the mgnification maps are:
* `thin_disk` (default option) more information available [here](https://arxiv.org/pdf/1707.01908.pdf).
* `sersic`
* `thin_disk&node`
* `wavy_hole`
* `wavy`
* `sersic`

In [49]:
def calR(x, xc, y, yc):
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def calXi(r, Rin, R0):
    """
    Function checks the value of r and calculates xi accordingly
    """
    if r<Rin:
        return 0
    else:
        if Rin == 0:
            xi = (r / R0) ** (3/4) * (1) ** (-1/4)
        else:
            xi = (r/R0)**(3/4)*(1-np.sqrt(Rin/r))**(-1/4)
    return xi

def getprofilevalue(x, y, xc, yc, xn, yn, I0, R0, model, Rin):
    """
    Function to get the profile values for each particular choice of model 
    """
    n, reff_pix, sersic_index =3, 0.2, 4.0
    beta = n*np.pi/(2*R0)
    
    if model == "thin_disk":
      r = calR(x, xc, y, yc)
      xi = calXi(r, Rin, R0)
      profile_val = I0/(np.exp(xi)-1)
        
    if model == "thin_disk&node":
      r = calR(x, xn, y, yn)
      xi = calXi(r, Rin, R0)
      profile_val = I0/(np.exp(xi)-1) + I0/(np.exp((r/(R0*0.4106))**(3/4))-1)

    if model =="wavy_hole":
      r = calR(x, xc, y, yc)
      # calculates the profile value based on conditions on beta*r
      if beta*r < 2*np.pi:
          profile_val = I0*beta/(n*np.pi**2)*np.power(np.sin(beta*r),2)/r
      else:
          profile_val = 0

    if model =="wavy":
      r = calR(x, xc, y, yc)
      if beta*r < np.pi/2:
          profile_val = I0*beta/(n*np.pi**2)
      elif beta * r < 2 * np.pi and beta * r > np.pi/2:
          profile_val = I0*beta/(n*np.pi**2)*np.square(np.sin(beta*r))
          
    if model == "sersic":
      r = calR(x, xc, y, yc)
      profile_val = I0 * np.exp(-(r / reff_pix) ** (1.0 / sersic_index))
    return profile_val

def generate2DDataArrays(dims=128):
    """
    Function to generate the 2D data arrays as per required dims
    @dims :: sets the size of the 2D data arrays
    """    
    data = np.zeros((128,128))
    pdata = np.zeros((128,128))
    gdata = np.zeros((128,128))
    return data, pdata, gdata

def get2ddiracvalue(x,y,xc,yc):
    """
    Function to get the 2D dirac values to fill the gaussian ???
    """
    if x == xc and y ==yc:
        return 1
    else:
        return 0
    
def outputMapParams(img):
    map_d = img.data[:, :]
    macro_mag = np.mean(map_d)
    map_d = map_d / macro_mag
    return map_d, macro_mag

def convolve(R0, map_name, model="thin_disk", Rin=0, I0=1):
    """
    Input:
    @map_name :: fits file of the magnification map
    @model :: the model of the light source that the magnification map will be convoluted with.
    @R0 :: float; scale radius, for thin_disk, R0 is R0 (~10^14 cm for 1131 & 0435), for sersic R0 is fwhm. (Units: pixels)
    @Rin :: float. Useful only for thin_disk
    @I0 :: float. Useful for both models
    #create source profile

    """
    img_name = map_name.split('map')[2].split('.')[0]
    img = fits.open(map_name)[0]
    map_d, macro_mag  = outputMapParams(img)

    xc, yc, xn, yn = 256, 256, 128, 128

    # generates the profile, gaussian, and toconv fits files from new_canvas.fits
    data, pdata, gdata = generate2DDataArrays(dims=128)

    # fills the data arrays based of the choice of the model
    for lind, line in enumerate(gdata):
        for cind, elt in enumerate(line):
            gdata[lind][cind] = get2ddiracvalue(cind+1, lind+1, xn, yn)
            pdata[lind][cind] = getprofilevalue(cind+1, lind+1, xc, yc, xn, yn, I0, R0, model, Rin)
            
    # convolves the data
    output = convolve_fft(gdata, pdata)

    for lind, line in enumerate(data):
        for cind, elt in enumerate(line):
            data[lind][cind] = out[lind][cind]
            
    # ???
    out2 = ss.fftconvolve(map_d, data, mode="valid")
    hdu = fits.PrimaryHDU(out2)

    hdu.writeto(storagedir+'/convolved_map_%s_fft_%s_%i_fml09.fits'%(img_name,model,R0))
    return

def getFilename(rootdir, string_name, params, param_arr=True):
    if param_arr:
        filename1 = os.path.join(rootdir, string_name%(params[0], params[2]))
        filename2 = os.path.join(rootdir, string_name%(params[1], params[2]))
        return filename1, filename2
    else:
        return os.path.join(rootdir, string_name%(params))

### 4. Execution of convolving the maps and save the data

In [50]:
for img_AB in list_img:
    for r0 in list_r0:
        print(r0)
        filename = getFilename(mapdir, 'map%s.fits', img_AB, param_arr=False)
        convolve(r0, filename,"thin_disk")

string_name = "/convolved_map_%s_fft_thin_disk_%s.fits"
for comb in list_comb:
    for r0 in list_r0:
        print(elem)
        params = [comb[0], comb[1], elem]
        filename1, filename2 = getFilename(storagedir, string_name, params, param_arr=True)
        final_map = mapDiff(filename1, filename2, elem, comb)

2


Exception: The kernel can't be normalized, because its sum is close to zero. The sum of the given kernel is < 0.01